# 2.3 运用数据库存储消息

我们的留言板虽然可以使用了,但因为是由程序中的全局变量维护的数据,所以一旦关闭程序我们的留言板就会被清空.要解决这个问题我们需要使用数据库.在之前的数据库部分我们已经了解过SQLAlchemy,flask有个插件叫[flask-SQLAlchemy](http://www.pythondoc.com/flask-sqlalchemy/quickstart.html),我们打算用它来实现app的数据库操作

我们只需要修改app.py和设置文件即可.

我们呢首先要哦通过命令行把表建出来,所以也要修改manager.py

In [25]:
%%writefile ../codes/msgboard/config.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals

import os
basedir = os.path.abspath(os.path.dirname(__file__))

class Config:
    SECRET_KEY = os.environ.get('SECRET_KEY') or 'hard to guess string' 
    SQLALCHEMY_COMMIT_ON_TEARDOWN = True  
    @staticmethod
    def init_app(app):
        pass
    

class DevelopmentConfig(Config): 
    SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'data.sqlite')#数据库位置
    DEBUG = True
         
class TestingConfig(Config): 
    TESTING = True
    
class ProductionConfig(Config):
    pass

config = {
    'development': DevelopmentConfig,
    'testing': TestingConfig, 
    'production': ProductionConfig,
    'default': DevelopmentConfig
}

    
    

Overwriting ../codes/msgboard/config.py


In [27]:
%%writefile ../codes/msgboard/app.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals
"""
A message board appliation.

Author:Huang Sizhe
Date:22/01/2016
License:MIT
======================================

留言板应用

作者:黄思喆
日期:2016年1月22日
本应用使用MIT许可证

"""
#=================导入模块=================
from flask import Flask,render_template,make_response,redirect,url_for

from flask.ext.bootstrap import Bootstrap

from flask_wtf.csrf import CsrfProtect
from flask.ext.wtf import Form
from wtforms import StringField, SubmitField 
from wtforms.validators import Required

from flask.ext.sqlalchemy import SQLAlchemy#导入ORM模块

#=================载入插件=================
bootstrap = Bootstrap()
csrf = CsrfProtect()
db = SQLAlchemy()# 实例化ORM对象
#=================应用设置=================
from config import config
def create_app(config_name):
    app = Flask(__name__)
    app.config.from_object(config[config_name])
    config[config_name].init_app(app)
    bootstrap.init_app(app)
    csrf.init_app(app)
    db.init_app(app)#初始化数据库
    return app

import os
app = create_app(os.getenv('FLASK_CONFIG') or 'default')    

#================主体=====================

#-----------------数据库对象---------------
class MSG(db.Model):
    __tablename__ = 'msgs'
    id = db.Column(db.Integer, primary_key=True)
    contents = db.Column(db.Text, unique=True)
    author = db.Column(db.String(64), unique=True)

    def __repr__(self):
        return '<Msg {} {}>'.format(self.author,self.contents)



##----------------表单--------------------

class MsgForm(Form):
    name = StringField('Your name ?', validators=[Required()])
    msg = StringField('The msg', validators=[Required()])
    submit = SubmitField('Submit')


##----------------view--------------------

@app.route('/',methods = ["GET","POST"])
def index():
    msgform = MsgForm()
    if msgform.validate_on_submit():
        
        msg = MSG(contents = msgform.msg.data,author = msgform.name.data)
        db.session.add(msg)
        db.session.commit()
        msgform.msg.data = ''
        msgform.name.data = ''
        return redirect(url_for('index'))
        MSG.query.all()
    response = make_response(render_template('app/index.html',
                                             msgform=msgform,
                                             MSG = MSG.query.all()))
    return response





Overwriting ../codes/msgboard/app.py


In [30]:
%%writefile ../codes/msgboard/templates/app/index.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}MSG BOARD{% endblock %}
{% block content %}

    <div class="container">
        <div class="page-header">
            <h1>MSG BOARD!</h1>
            {{ wtf.quick_form(msgform,form_type='horizontal') }}
            
            <ul class=entries>
              {% for entry in MSG %}
                <li><h4>{{ entry.author  }} say:</h4><p>{{ entry.contents }}<p>
              {% else %}
                <li><em>Unbelievable.  No entries here so far</em>
              {% endfor %}
            </ul>
            
        </div>      
    </div>    
    
{{ super() }}
{% endblock %}

Overwriting ../codes/msgboard/templates/app/index.html


### 将数据库集成到shell

每次启动 shell 会话都要导入数据库实例和模型,这真是份枯燥的工作。为了避免一直重复
导入,我们可以做些配置,让 Flask-Script 的 shell 命令自动导入特定的对象。 若想把对象添加到导入列表中,我们要为 shell 命令注册一个 make_context 回调函数在其中注册好需要的变量即可,要在shell中操作数据库只要在再后面添加一句
`manager.add_command("shell", Shell(make_context=make_shell_context))`,运行的时候用shell作为参数即可

In [26]:
%%writefile ../codes/msgboard/manager.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals
"""
A startup manager of the application.

Author:Huang Sizhe
Date:22/01/2016
License:MIT
======================================

应用的启动文件

作者:黄思喆
日期:2016年1月22日
本应用使用MIT许可证

"""

__author__ = "Huang Sizhe"
__date__ = "22/01/2016"

import os
import sys
from flask.ext.script import Manager,Shell

root = os.path.dirname(__file__)

from app import app,db,MSG

manager = Manager(app)

def make_shell_context():
    return dict(app=app, db=db, MSG=MSG)
manager.add_command("shell", Shell(make_context=make_shell_context))

if __name__ == '__main__':
    manager.run()

Overwriting ../codes/msgboard/manager.py


在自己的terminal里通过manager.py启动shell

    
    python3 ../codes/msgboard/manager.py shell
    
    
进去后创建表

    db.create_all()

In [31]:
!python3 ../codes/msgboard/manager.py runserver

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/flask_sqlalchemy/__init__.py:800: UserWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.
  warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.')
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/flask_sqlalchemy/__init__.py:800: UserWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.
  warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.')
 * Debugger is active!
 * Debugger pin code:

# 总结 

用到的模块:

包|作用
---|---
flask|flask web框架
flask-script|flask的上下文shell
jinja2|flask的默认模板
flask-bootstrap|flask的bootstrap前端扩展
flask-wtf|构建表单
flask-SQLAlchemy|关系数据库ORM